In [1]:
import sys
sys.path.append('../../src/')
from typing import Union, Sequence, Optional

import numpy as np
from qiskit import *
from ansatze import build_ne2_ansatz
from hamiltonians import MolecularHamiltonian
from qiskit.utils import QuantumInstance
from greens_function import GreensFunction
from tools import get_quantum_instance
from constants import HARTREE_TO_EV
from qiskit.opflow import PauliSumOp, PrimitiveOp, Z2Symmetries
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Pauli, PauliTable
from tools import number_state_eigensolver
from z2_symmetries import apply_cnot_z2

/home/snsunx/.local/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


ImportError: cannot import name 'number_state_eigensolver' from 'tools' (/mnt/c/Users/snsun/Documents/Code/quantum-simulation/greens-function/tests/lih/../../src/tools.py)

In [6]:
ansatz = build_ne2_ansatz(4)
hamiltonian = MolecularHamiltonian(
    [['Li', (0, 0, 0)], ['H', (0, 0, 1.6)]], 'sto3g', 
    occupied_inds=[0], active_inds=[1, 2])
qiskit_op = hamiltonian.qiskit_op
number_state_eigensolver(hamiltonian, 1)[0] * HARTREE_TO_EV

array([-206.32171881, -206.32171881, -194.6801225 , -194.6801225 ])

In [7]:
def apply_cnot_z2(pauli_sum_op: Union[PauliSumOp, SparsePauliOp], 
                  ctrl: int, 
                  targ: int) -> PauliSumOp:
    if isinstance(pauli_sum_op, PauliSumOp):
        prim_op = pauli_sum_op.primitive.copy()
    elif isinstance(pauli_sum_op, SparsePauliOp):
        prim_op = pauli_sum_op.copy()
    else:
        raise TypeError
    coeffs = prim_op.coeffs.copy()
    x_arr = prim_op.table.X.copy()
    z_arr = prim_op.table.Z.copy()
    n_paulis, n_qubits = x_arr.shape
    
    # Flip sign when the pauli is X_cZ_t or Y_cY_t
    for i in range(n_paulis):
        x = x_arr[i]
        z = z_arr[i]
        if z[targ] == x[ctrl] == True and z[ctrl] == x[targ]:
            coeffs[i] *= -1
    
    x_arr[:, targ] ^= x_arr[:, ctrl] # forward on X part    
    z_arr[:, ctrl] ^= z_arr[:, targ] # backward on Z part
    
    sparse_pauli_op = SparsePauliOp(np.hstack((x_arr, z_arr)), coeffs=coeffs)
    pauli_sum_op_new = PauliSumOp(sparse_pauli_op)
    return pauli_sum_op_new

def taper(pauli_sum_op: Union[PauliSumOp, SparsePauliOp], 
          inds_tapered: Sequence[int],
          init_state: Optional[Sequence[int]] = None) -> PauliSumOp:
    if isinstance(pauli_sum_op, PauliSumOp):
        prim_op = pauli_sum_op.primitive.copy()
    elif isinstance(pauli_sum_op, SparsePauliOp):
        prim_op = pauli_sum_op.copy()
    else:
        raise TypeError
    coeffs = prim_op.coeffs.copy()
    x_arr = prim_op.table.X.copy()
    z_arr = prim_op.table.Z.copy()
    n_paulis, n_qubits = x_arr.shape
    n_tapered = len(inds_tapered)
    if init_state is None:
        init_state = [0] * n_tapered

    for i in range(n_paulis):
        z = z_arr[i]
        # print(prim_op.table[i])
        for j in range(n_tapered):
            # print("j =", j)
            # print(z[n_qubits - 1 - j])
            if z[j] == True and init_state[j] == 1: # Is this general?
                coeffs[i] *= -1
                # print(f"{prim_op.table[i]} multiplied by -1")

    inds_kept = sorted(set(range(n_qubits)) - set(inds_tapered))
    inds_kept = sorted([n_qubits - 1 - i for i in inds_kept]) # due to Qiskit qubit order
    x_arr = x_arr[:, inds_kept]
    z_arr = z_arr[:, inds_kept]
    
    sparse_pauli_op = SparsePauliOp(np.hstack((x_arr, z_arr)), coeffs=coeffs)
    pauli_sum_op_new = PauliSumOp(sparse_pauli_op)
    return pauli_sum_op_new

In [8]:
qiskit_op_new = apply_cnot_z2(apply_cnot_z2(qiskit_op, 2, 0), 3, 1)
qiskit_op_tapered_00 = taper(qiskit_op_new, [2, 3], init_state=[0, 0])
qiskit_op_tapered_01 = taper(qiskit_op_new, [2, 3], init_state=[0, 1])
qiskit_op_tapered_10 = taper(qiskit_op_new, [2, 3], init_state=[1, 0])
qiskit_op_tapered_11 = taper(qiskit_op_new, [2, 3], init_state=[1, 1])

In [201]:
qiskit_op_arr = qiskit_op.to_matrix() * HARTREE_TO_EV
qiskit_op_tapered_00_arr = qiskit_op_tapered_00.to_matrix() * HARTREE_TO_EV
qiskit_op_tapered_01_arr = qiskit_op_tapered_01.to_matrix() * HARTREE_TO_EV
qiskit_op_tapered_10_arr = qiskit_op_tapered_10.to_matrix() * HARTREE_TO_EV
qiskit_op_tapered_11_arr = qiskit_op_tapered_11.to_matrix() * HARTREE_TO_EV

eigvals = np.linalg.eigh(qiskit_op_arr)[0]
eigvals_tapered_00 = np.linalg.eigh(qiskit_op_tapered_00_arr)[0]
eigvals_tapered_01 = np.linalg.eigh(qiskit_op_tapered_01_arr)[0]
eigvals_tapered_10 = np.linalg.eigh(qiskit_op_tapered_10_arr)[0]
eigvals_tapered_11 = np.linalg.eigh(qiskit_op_tapered_11_arr)[0]
print(eigvals)
print(eigvals_tapered_00)
print(eigvals_tapered_01)
print(eigvals_tapered_10)
print(eigvals_tapered_11)

[-213.9394244  -211.80955735 -211.80955735 -210.12598445 -210.12598445
 -210.12598445 -209.73727191 -206.32171881 -206.32171881 -204.526909
 -204.526909   -200.48123171 -194.99412728 -194.6801225  -194.6801225
 -185.14660667]
[-210.12598445 -210.12598445 -200.48123171 -185.14660667]
[-211.80955735 -206.32171881 -204.526909   -194.6801225 ]
[-211.80955735 -206.32171881 -204.526909   -194.6801225 ]
[-213.9394244  -210.12598445 -209.73727191 -194.99412728]


In [10]:
print(qiskit_op_new[:5])

-7.509157198656697 * IIII
+ 0.1559240918074194 * IZIZ
- 0.014015945283725094 * ZXZZ
+ 0.014015945283725094 * ZXZI
+ 0.1559240918074194 * ZIZI


In [119]:
print(qiskit_op_tapered)

-7.509157198656667 * II
+ 0.15592409180744818 * IZ
- 0.014015945283564694 * ZZ
+ 0.014015945283564694 * ZI
+ 0.1559240918074481 * ZI
- 0.014015945283564694 * ZI
+ 0.014015945283564694 * II
- 0.0150398228134703 * II
- 0.015039822813470303 * II
+ 0.12182774215821793 * ZZ
- 0.012144897228058425 * IZ
+ 0.012144897228058425 * II
- 0.012144897228058425 * ZZ
+ 0.012144897228058425 * IZ
+ 0.003265995499655414 * IZ
- 0.003265995499655414 * ZZ
- 0.003265995499655414 * II
+ 0.003265995499655414 * ZI
+ 0.052636515285500896 * IZ
+ 0.055902510785156304 * IZ
+ 0.0018710427514141413 * ZZ
- 0.0018710427514141413 * ZI
+ 0.055902510785156304 * ZI
+ 0.0018710427514141413 * ZI
- 0.0018710427514141413 * II
+ 0.052636515285500896 * ZI
+ 0.08447056807294723 * II


In [26]:
print(Pauli('IX').x)

[ True False]


In [23]:
sparse_pauli_op = SparsePauliOp(PauliTable.from_labels(['ZX', 'ZZ', 'YY', 'XX']))
sparse_pauli_op[3].table.array.shape[0]

1

In [24]:
sparse_pauli_op[3].coeffs

array([1.+0.j])

In [190]:
PauliSumOp(sp)

PauliSumOp(SparsePauliOp([[False, False, False, False, False, False, False, False],
               [False, False, False, False,  True, False, False, False],
               [ True, False,  True, False,  True,  True,  True, False],
               [ True, False,  True, False, False,  True, False, False],
               [False, False, False, False, False,  True, False, False],
               [False,  True, False,  True, False,  True,  True,  True],
               [False,  True, False,  True, False, False,  True, False],
               [False, False, False, False, False, False,  True, False],
               [False, False, False, False, False, False, False,  True],
               [False, False, False, False,  True,  True, False, False],
               [ True, False,  True, False,  True, False,  True, False],
               [ True, False,  True, False, False, False, False, False],
               [False,  True, False,  True,  True,  True,  True,  True],
               [False,  True, False,  Tr

In [5]:
x_ops = SparsePauliOp(PauliTable.from_labels(['IIIX', 'IIXZ', 'IXZZ', 'XZZZ']))
y_ops = SparsePauliOp(PauliTable.from_labels(['IIIY', 'IIYZ', 'IYZZ', 'YZZZ']))

In [6]:
x_ops_trans = apply_cnot_z2(apply_cnot_z2(x_ops, 2, 0), 3, 1).primitive.table.to_labels()
y_ops_trans = apply_cnot_z2(apply_cnot_z2(y_ops, 2, 0), 3, 1).primitive.table.to_labels()
print(x_ops_trans)
print(y_ops_trans)

['IIIX', 'IZXZ', 'ZYZY', 'YIYZ']
['IZIY', 'ZZYZ', 'ZXZY', 'XIYZ']


In [2]:
table = PauliTable.from_labels(['ZX'])

In [3]:
table.X

array([[ True, False]])

In [4]:
table.Z

array([[False,  True]])

In [12]:
import numpy as np
Pauli('ZIZI').to_matrix()

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0

In [13]:
from tools import *

In [30]:
inds = get_number_state_indices(4, 2)
print(inds)

[3, 5, 6, 9, 10, 12]


In [31]:
inds_rev = [15 - i for i in inds]
print(inds_rev)

[12, 10, 9, 6, 5, 3]


In [33]:
circ = QuantumCircuit(3)
circ.cp(3.2, 0, 1)

In [34]:
print(circ)

              
q_0: ─■───────
      │P(3.2) 
q_1: ─■───────
              
q_2: ─────────
              


In [41]:
a = 'II'
b = a

In [42]:
a = a + 'X'

In [43]:
a

'IIX'

In [44]:
b

'II'

In [46]:
from cmath import polar

In [47]:
polar(1j)

(1.0, 1.5707963267948966)

In [49]:
set(list('IIIII')) == {'I'}

True